In [ ]:
import pandas as pd
from khaiii import KhaiiiApi
import re

url = "https://raw.githubusercontent.com/JunHCha/K-POP-Lyrics-1964-2020/main/data/lyrics_original/lyric_preprocessed_unique.csv"
lyrics = pd.read_csv(url, encoding="utf-8")


In [ ]:
api = KhaiiiApi()

word_df = pd.DataFrame(
    columns=["genre", "year", "song_id", "title", "line", "word", "class", "tag_orig"]
)

for i, each in lyrics.iterrows():
    genre = each["genre"]
    year = each["year"]
    song_id = each["song_id"]
    title = each["title"]
    print(year, title, sep=" ")

    # 개행문자 단위로 가사를 문장으로 분리
    lines = re.split("\n", each["lyric"])

    # 두 번 개행한 경우 제거
    lines = list(filter(lambda x: x != "", lines))

    # 한 문장마다 Khaiii 형태소 분석
    for i, line in enumerate(lines):
        try:
            analyzed = api.analyze(line)
        except :
            continue

        for word in analyzed:
            for morph in word.morphs:
                if morph.tag == "NNG" or morph.tag == "NNP":
                    word_class = "Noun"
                elif morph.tag == "VA":
                    word_class = "Adjective"
                else:
                    continue

                line_word = pd.DataFrame(
                    {
                        "genre": [genre],
                        "year": [year],
                        "song_id": [song_id],
                        "title": [title],
                        "line": [i],
                        "word": [morph.lex],
                        "class": [word_class],
                        "tag_orig": [morph.tag]
                    }
                )
                word_df = pd.concat([word_df, line_word])
word_df

In [ ]:
word_df.to_csv("../data/lyrics_original/lyric_word_parsed_by_line.csv")